<a href="https://colab.research.google.com/github/j-innovatech/Alliance_des_10_provinces/blob/main/Data_cleaning_Alliance_des_10_provinces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data cleaning for Alliance des 10 provinces dataset
==============

This notebook process the data from Heurist for further analysis of in J-innvovatech's Salt Alliance project.

This is for a publication proposal for a data paper on ...

Author:
Hiroki Yamashita (Sciences Po)

This code is constructed in the following way.

1. Load two json files from the Heurist database. One for meetings and one for actors.
2. Split raws of the meeting data so that each raw contains only one actor
3. Merge data with actor data so that thier Japanese names as well as domains and provinces are in the data.
4. Calculate the number of the time an actor had been to meetings and make a column
5. Save the new dataframe in CSV format

In [1]:
# Import necessary packages
import pandas as pd
import re
import json

## Load data
The following block loads the data from the github repository of J-innovatech at the following adress:
https://github.com/j-innovatech/Alliance_des_10_provinces


In [ ]:
! git clone https://github.com/j-innovatech/Alliance_des_10_provinces
path1 = "./Alliance_des_10_provinces/data/Meetings.json"
path2 = "./Alliance_des_10_provinces/data/Actors.json"

In [3]:
#Place of Meetings
DictMeetingPlace={
    "An.yōji - 安養寺":"Aki",
    "Itsukushima - 厳島":"Aki",
    "Yuga-san - 瑜伽山（由加山）":"Bizen",
    "Marugame - 丸亀":"Sanuki"
}

In [7]:
# Creating a dataframe for meetings from json file

# Opening JSON file
f = open(path1)
data = json.load(f)

# Columns for the dataframe
Meetings=[]
Actors=[]
DatesJP = [] #field name "Date(s)"
Datedebut=[] #field name "Date greg"
Datefin=[] #field name "Date greg"
DateYears=[]
MeetingPlacesFR = [] #field name "Lieux de réunion"
MeetingPlacesJP = [] #field name "Lieux de réunion"
MeetingProvinces =[]
Sources=[] #field name "Sources"

#Loop for each meeting
for meeting in data["heurist"]["records"]:
  MeetingParticipants=[]
  #Erasing the indent, \n
  MeetingsWithoutIndent =re.sub("[\n]+"," ",meeting["rec_Title"])
  Meetings.append(MeetingsWithoutIndent)

  #Loop for additional information on meeting
  for detail in meeting["details"]:
    if detail["fieldName"]=="Acteur":
      #Erasing the indent, \n
      NameWithoutIndent =re.sub("[\n]+"," ",detail["value"]["title"])
      MeetingParticipants.append(NameWithoutIndent)

    #"Date(s)"
    if detail["fieldName"]=="Date(s)":
      #Erasing the indent, \n
      DateJPWithoutIndent =re.sub("[\n]+"," ",detail["value"]["title"])
      DatesJP.append(DateJPWithoutIndent)

    #"Date greg"
    if detail["fieldName"]=="Date greg":
      if isinstance(detail["value"],dict):
        Datedebut.append(detail["value"]["start"]["earliest"])
        Datefin.append(detail["value"]["end"]["latest"])
        DateYear = re.findall(r"([0-9]{4})",detail["value"]["start"]["earliest"])[0]
        DateYears.append(DateYear)

      else:
        Datedebut.append(detail["value"])
        Datefin.append(detail["value"])
        DateYear = re.findall(r"([0-9]{4})",detail["value"])[0]
        DateYears.append(DateYear)

    #"Lieux de réunion"
    if detail["fieldName"]=="Lieux de réunion":
      MeetingPlacesFR.append(detail["value"]["title"].split(" - ")[0])
      MeetingPlacesJP.append(detail["value"]["title"].split(" - ")[1])
      MeetingProvinces.append(DictMeetingPlace[detail["value"]["title"]])

    #"Sources"
    if detail["fieldName"]=="Sources":
      Sources.append(detail["value"]["title"])

  #Append the list of participants for a meeting
  Actors.append(MeetingParticipants)

df = pd.DataFrame(list(zip(Actors,Meetings,DatesJP,DateYears,Datedebut,Datefin,MeetingPlacesFR,MeetingPlacesJP,MeetingProvinces,Sources)),
               columns =['Acteur', "Description événement","Date JP","Année","Date de debut greg","Date de fin greg","Lieux de réunion FR","Lieux de réunion JP","Province de réunion","Sources"])

#Split raws by actors
df =df.explode("Acteur",ignore_index=True)

#close the json file
f.close()

In [8]:
# Creating a dataframe for actors from json file

# Opening JSON file of actors
f = open(path2)
data = json.load(f)

# List of actors who went to at least one meeting
Actorslist=list(df["Acteur"].value_counts().index)

# Columns for the dataframe
Actors=[]
Domains=[] #field name "Province ou domaine associé"
Provinces=[] #field name "Province ou domaine associé"
NameFR =[] #field name "Nom acteur fr"
NameJP = [] #field name "Nom acteur JP"
StatusFR1=[]#field name "Statut de l'acteur"
StatusJP1=[]#field name "Statut de l'acteur"
StatusFR2=[]#field name "Statut de l'acteur"
StatusJP2=[]#field name "Statut de l'acteur"
AssociatedSitesFR1 = [] #field name "Site associé"
AssociatedSitesJP1= [] #field name "Site associé"
AssociatedSitesFR2 = [] #field name "Site associé"
AssociatedSitesJP2 = [] #field name "Site associé"
AssociatedSitesFR3 = [] #field name "Site associé"
AssociatedSitesJP3= [] #field name "Site associé"

#Loop each actor
for actor in data["heurist"]["records"]:
  #Erasing the indent, \n
  NameWithoutIndent =re.sub("[\n]+"," ",actor["rec_Title"])

  #Put actor data only if the actor has attended to the meeting in the meeting list
  if NameWithoutIndent in Actorslist:
    Actors.append(NameWithoutIndent)
    #Counter for the number of associated sites and status
    Sitecount=0
    Statuscount=0

    for detail in actor["details"]:
      #"Province ou domaine associé"
      if detail["fieldName"]== "Province ou domaine associé":
        Domains.append(re.findall(r"Hiroshima|Tokushima|Okayama|Hagi|Himeji|Ehime|Iwakuni|Matsuyama|Marugame|Takamatsu|Akō",detail["value"])[0])
        Provinces.append(re.findall(r"Bizen|Iyo|Suō|Awa|Aki|Bizen|Bingo|Settsu|Nagato|Harima|Awaji|Sanuki|Tosa",detail["value"])[0])

      #'Nom acteur fr'
      if detail["fieldName"]== "Nom acteur fr":
        NameFR.append(detail["value"])

      #'Nom acteur JP'
      if detail["fieldName"] == "Nom acteur jap":
        NameJP.append(detail["value"])

      #'Statut de l'acteur'
      if detail["fieldName"] == "Statut de l'acteur":
        Statuscount+=1

        if Statuscount== 1:
          StatusFR1.append(detail["value"]["title"].split(" - ")[0])
          StatusJP1.append(detail["value"]["title"].split(" - ")[1])
        if Statuscount== 2:
          StatusFR2.append(detail["value"]["title"].split(" - ")[0])
          StatusJP2.append(detail["value"]["title"].split(" - ")[1])

      #'Site associé'
      if detail["fieldName"] == "Site associé":
        Sitecount+=1

        # Put the name of the production site depending on the number of sites for an actor
        if Sitecount== 1:
          AssociatedSitesFR1.append(detail["value"]["title"].split(" - ")[0])
          AssociatedSitesJP1.append(detail["value"]["title"].split(" - ")[1])
        if Sitecount== 2:
          AssociatedSitesFR2.append(detail["value"]["title"].split(" - ")[0])
          AssociatedSitesJP2.append(detail["value"]["title"].split(" - ")[1])
        if Sitecount== 3:
          AssociatedSitesFR3.append(detail["value"]["title"].split(" - ")[0])
          AssociatedSitesJP3.append(detail["value"]["title"].split(" - ")[1])

    # Put a blank for status and production site when there is no record
    if Statuscount==0:
      StatusFR1.append("")
      StatusJP1.append("")
      StatusFR2.append("")
      StatusJP2.append("")
    if Statuscount==1:
      StatusFR2.append("")
      StatusJP2.append("")

    if Sitecount== 0:
      AssociatedSitesFR1.append("")
      AssociatedSitesJP1.append("")
      AssociatedSitesFR2.append("")
      AssociatedSitesJP2.append("")
      AssociatedSitesFR3.append("")
      AssociatedSitesJP3.append("")

    if Sitecount== 1:
      AssociatedSitesFR2.append("")
      AssociatedSitesJP2.append("")
      AssociatedSitesFR3.append("")
      AssociatedSitesJP3.append("")

    if Sitecount== 2:
      AssociatedSitesFR3.append("")
      AssociatedSitesJP3.append("")

Actor_df = pd.DataFrame(list(zip(Actors,NameFR,NameJP,Domains,Provinces,StatusFR1,StatusJP1,StatusFR2,StatusJP2,AssociatedSitesFR1,AssociatedSitesJP1,AssociatedSitesFR2,AssociatedSitesJP2,AssociatedSitesFR3,AssociatedSitesJP3)),
                columns =['Acteur', "Nom acteur FR","Nom acteur JP","Domain","Province","Statut de l'acteur FR 1","Statut de l'acteur JP 1","Statut de l'acteur FR 2","Statut de l'acteur JP 2","Site associé FR 1","Site associé JP 1","Site associé FR 2","Site associé JP 2","Site associé FR 3","Site associé JP 3"])

f.close()

In [9]:
# Merge datafrane for meetings and actors
df = df.merge(Actor_df,left_on ="Acteur",right_on="Acteur", how ="left")

In [10]:
# Create a new column with number of times an actor have been to a meeting
Actors=df["Acteur"].value_counts().index
N_participation=df["Acteur"].value_counts().values

df["Participation"] = 0
for i, actor in enumerate (Actors):
  df.sort_values(by="Année",inplace=True)
  df.loc[df["Acteur"]==actor,"Participation"]= range(1, N_participation[i] + 1)

# Create a new column for checking if the province of meeting and the actor is the same
df["Même province"] = False
df.loc[df["Province"]==df["Province de réunion"],"Même province"] = True

In [14]:
# Save the dataframe to CSV
df.to_csv("Alliance_des_10_provinces_cleaned.csv",index=False)

In [11]:
len(df)

473